## 3-D field scanner: Using a LED strip for calculation of the speed profile of the 3-D printer 

## Purpose
In this notebook, we will give a detailed explanation of how we got a speed profile of the 3-d printer. This was necessary for knowing where the hall sensor was at what time, which was important for getting an accurate picture of what the magnetic field looked like.
## General idea
We attach a light sensor to the arm of the 3-D printer and measure light at equal intervals on the z-axis. The light sensor then measures peaks at certain times and we can calculate a spedd profile of the arm.
## Steps
1. Build a row of lightsources at constant intervals
2. Explanation of code of LED strip
3. Explanation of GCode for 3-D printer

## Light source
To get light sources at constant intervals we used a PVC tube. We drilled holes with a diameter of approximately 1 mm. They need to be small so the location at which the light sensor measures light is small en you get an accurate representation of position to time. However do not make it too small otherwise you might get diffraction of the light, which only decreases the precision.
These holes where located at intervals of 1 cm. 
Inside the PVC tube we put a Neopixel WS2812b LED strip. Through an Arduino Uno WiFi we could determine the intensity and how many LEDs the strip had. In the next part we will explain the code we used.

Below is the code for the arduino. The only things you need to check is to which pin you need to cennect the lightstrip, which you can change by altering the number after #define PIN; and how many NeoPixels the lightstrip has, which you need to change by altering the number behind #define NUMPIXELS. The rest of the code is ready for use.

In [ ]:
// NeoPixel Ring simple sketch (c) 2013 Shae Erisson
// Released under the GPLv3 license to match the rest of the
// Adafruit NeoPixel library

#include <Adafruit_NeoPixel.h>
#ifdef __AVR__
 #include <avr/power.h> // Required for 16 MHz Adafruit Trinket
#endif

// Which pin on the Arduino is connected to the NeoPixels?
#define PIN        6

// How many NeoPixels are attached to the Arduino?
#define NUMPIXELS 30 

// When setting up the NeoPixel library, we tell it how many pixels,
// and which pin to use to send signals. Note that for older NeoPixel
// strips you might need to change the third parameter.
Adafruit_NeoPixel pixels(NUMPIXELS, PIN, NEO_GRB + NEO_KHZ800);

#define DELAYVAL 500 // Time (in milliseconds) to pause between pixels

void setup() {
  // These lines are specifically to support the Adafruit Trinket 5V 16 MHz.
  // Any other board, you can remove this part (but no harm leaving it):
#if defined(__AVR_ATtiny85__) && (F_CPU == 16000000)
  clock_prescale_set(clock_div_1);
#endif
  // END of Trinket-specific code.

  pixels.begin(); // INITIALIZE NeoPixel strip object (REQUIRED)
}

void loop() {

  // The first NeoPixel in a strand is #0, second is 1, all the way up
  // to the count of pixels minus one.
  for(int i=0; i<NUMPIXELS ; i++) { // For each pixel...

    // pixels.Color() takes RGB values, from 0,0,0 up to 255,255,255
    // Here we're using a bright white colour:
    pixels.setPixelColor(i, pixels.Color(255, 255, 255));

    pixels.show();   // Send the updated pixel colours to the hardware.

  }
}  

## 3-D printer
For measuring you also need a machine that can make controllable movements to which you can attach the light sensor. We used a FLsun Q5 3-D printer. However there were some changes made beforehand so we could use it as a controllable arm instead of a regular 3-D printer. The filament coil was removed and also the cooling fan on the arm was detached. When we received the printer we also removed the heeating plate on which you normally print so that we could later on measure the magnetic field below the printer using an extension of the arm, we will come back on this. Now we have a machine that we could use to make measurements. 

The printer is controlled using GCode. There are two ways of sending GCode to the printer: either with a micro SD card or using USB cables. We used the micro SD card. We put a GCode file on the memory card and after inserting the memory card on the printer you can start the file using the screen on the printer itself. The code will be explained next.

Since we attached both the light sensor and the Hall sensor to the wires which normally connect to the cooling fan, we activate the sensors by telling the printer to start the fan. If we want to start the measurement we turn on the "fan", wait a second, turn it off again and then start moving the arm. After we have moved the arm we stop the measurement again by turning it on and off again. This is all included in the GCode below. With this code the arm just goes straight up and down and takes a measurement while it is moving.

In [ ]:
G21 ; millimeters
G90 ; absolute coordinate
G17 ; XY plane
G94 ; units per minute feed rate mode
F2000 ; set feed rate slow for measurement (mm/min)
G28 ; Sends to home
M204 T3000 ; sets acceleration

; Create straight line through center
G0 X0 Y0 Z210 ; Goes to the top
G4 S5 ; Waits for 5 seconds
M106 S1000 ; Turns the fan on (Starts measurement)
G4 S1 ; Waits for 1 second
M107 ; Turns the fan off
G0 X0 Y0 Z0 ; Goes to the bottom
M106 S1000 ; Turns the fan on (Stops measurement)
G4 S1 ; Waits for 1 second
M107 ; Turns the fan off
G4 S5 ; Waits for 5 seconds
M106 S1000 ; Turns the fan on (Starts measurement)
G4 S1 ; Waits for 1 second
M107 ; Turns the fan off
G0 X0 Y0 Z210 ; Goes to the top
M106 S1000 ; Turns the fan on (Stops measurement)
G4 S1 ; Waits for 1 second
M107 ; Turns the fan off
G4 S5 ; Waits for 5 seconds
M106 S1000 ; Turns the fan on (Starts measurement)
G4 S1 ; Waits for 1 second
M107 ; Turns the fan off
G0 X0 Y0 Z0 ; Goes to the bottom
M106 S1000 ; Turns the fan on (Stops measurement)
G4 S1 ; Waits for 1 second
M107 ; Turns the fan off
G4 S5 ; Waits for 5 seconds
M106 S1000 ; Turns the fan on (Starts measurement)
G4 S1 ; Waits for 1 second
M107 ; Turns the fan off
G0 X0 Y0 Z210 ; Goes to the top
M106 S1000 ; Turns the fan on (Stops measurement)
G4 S1 ; Waits for 1 second
M107 ; Turns the fan off